# Working file : Network.xml

In [3]:
# coding: utf-8
import xml.etree.ElementTree as ETree
import pandas as pd
import json 
import utm

In [4]:
# Read xml
nodeData = '/Users/xinyun/Documents/Stage4A/data/Network.xml'
prstree = ETree.parse(nodeData)
root = prstree.getroot()

# *The endpoints of the simulation area that I used during my intern*

In [90]:
# contour
(xmin_simulation, ymin_simulation, xmax_simulation, ymax_simulation) = (625017.9576056905, 5798791.299367087,
                                                                        634641.3677946266, 5810286.725571055)


## **Get 2D list of node** 

In [78]:
# Create two lists
items = []
node = []

# Iterate through the root node to get the child node 
for n in root.iter('node') :
    id_nb = n.attrib.get('id')
    x = n.attrib.get('x')
    y = n.attrib.get('y')  
    items = [id_nb, x, y]   # Create a list for each row
    node.append(items)

### Number of nodes

In [79]:
len(node)

17435

### Get the endpoints of nodes

In [80]:
xmin = float(min(node[r][1] for r in range(len(node))))
ymin = float(min(node[r][2] for r in range(len(node))))
xmax = float(max(node[r][1] for r in range(len(node))))
ymax = float(max(node[r][2] for r in range(len(node))))

In [81]:
print(xmin, ymin, xmax, ymax)

594329.0264295612 5764460.89353358 652661.608364352 5843471.6513260715


In [82]:
# get the length of the node coverage area
xmax - xmin

58332.581934790825

In [83]:
# get the width of the node coverage area
ymax - ymin

79010.75779249147

## **Get 2D list of link** 

In [84]:
# Create two lists
items_ = []
link = []
car = []
c = []
# Iterate through the root node to get the child node
for l in root.iter('link') :
    id_nb = l.attrib.get('id')
    start = l.attrib.get('from')
    end = l.attrib.get('to')
    length = l.attrib.get('length')
    modes = l.attrib.get('modes')
    items_ = [id_nb, start, end, length, modes]
    # List contains all kinds of modes of transport (car, pt)
    link.append(items_)
    # List contains only cars(car and pt,car)
    if (modes == 'pt,car') | (modes == 'car') :
        c = [id_nb, start, end, length, 'CAR']
        car.append(c)      

### Number of links

In [86]:
len(link)

31177

## **Dictionary and Json file**

### Initialisation 

In [91]:
# Empty nested dictionary
Network = { 'ROADS': {'NODES': {}, 'STOPS': {}, 'SECTIONS': {}, 'ZONES': {} },
         'LAYERS': [{}], 'TRANSIT': [] }

In [92]:
# Initialisation the first position of 'LAYERS'
Network['LAYERS'][0] = {'ID' : 'CAR', 'TYPE' : 'mnms.graph.layers.CarLayer',  'VEH_TYPE': 'mnms.vehicles.veh_type.Car',
    'DEFAULT_SPEED': 13.8,  'SERVICES': [], 'NODES': [], 'LINKS': [], 'MAP_ROADDB': {'NODES' : {}, 'LINKS' : {}}}

### Creat network Json file

In [93]:

for part,info in Network.items() :
    # the ROADS object
    if part == 'ROADS' :
        for key in info :
            if key == 'NODES' :  # Nodes part
                for i in range(len(node)) :
                    id_node = node[i][0]
                    x = float(node[i][1])
                    y = float(node[i][2])
                    info[key][id_node] = {}
                    info[key][id_node]['id'] = id_node
                    info[key][id_node]['position'] = [x,y]
            if key == 'SECTIONS' : # Sections part
                for i in range(len(link)) :
                    id_section = link[i][0]
                    upstream = link[i][1]
                    downstream = link[i][2]
                    length = float(link[i][3])
                    zone = 'RES'
                    info[key][id_section] = {'id' : id_section, 'upstream' : upstream, 'downstream' : downstream, 'length' : length, 'zone' : zone}
               # Stops part ......
            if key == 'ZONES' : # Zones part
                info[key]['RES'] = {}
                info[key]['RES']['id'] = 'RES'
                info[key]['RES']['sections'] = []
                # add the endpoints of the simulation area to the contour
                info[key]['RES']['contour'] = [[xmin_simulation, ymin_simulation],[xmin_simulation, ymax_simulation], 
                                               [xmax_simulation, ymin_simulation], [xmax_simulation, ymax_simulation]]
                for i in range(len(link)) :
                    id_section = link[i][0]
                    info[key]['RES']['sections'].append(id_section)
    # the LAYERS object
    if part == 'LAYERS' :
        # Car layer (first {})
        for key,value in  Network['LAYERS'][0].items() :
            if key == 'NODES' :
                for i in range(len(node)) :
                    ID = node[i][0]
                    X = float(node[i][1])
                    Y = float(node[i][2])
                    value.append({'ID' : ID, 'X' : X, 'Y' : Y, 'LABEL' : 'CAR', 'EXCLUDE_MOVEMENTS' : {}})
            if key == 'LINKS' :
                for i in range(len(car)) :
                    ID = car[i][0]
                    upstream = car[i][1]
                    downstream = car[i][2]
                    length = float(car[i][3])
                    label = car[i][4]
                    value.append({'ID' : ID, 'UPSTREAM': upstream, 'DOWNSTREAM' : downstream, 'COSTS' : {'length' : length}, 'LABEL' : label})
            if key == 'MAP_ROADDB' :
                for k,v in Network['LAYERS'][0]['MAP_ROADDB'].items() :
                    # Key = 'NODES'
                    if k == 'NODES' :
                        for i in range(len(node)) :
                            ID_node = node[i][0]  # Get node ID
                            v[ID_node] = ID_node
                    # Key = 'LINKS'
                    if k == 'LINKS' :
                        for i in range(len(car)) :
                            ID_link = car[i][0] # Get link ID
                            v[ID_link] = [ID_link]
                            
  #  if part == 'TRANSIT' :
   #     for i in range(len(car)) :
    #        ID = car[i][0]
     #       upstream = car[i][1]
      #      downstream = car[i][2]
      #      length = float(car[i][3])
       #     label = car[i][4]
        #    info.append({'ID' : ID, 'UPSTREAM': upstream, 'DOWNSTREAM' : downstream, 'LENGTH' : length,  'COSTS' : {'length' : length}, 'LABEL' : label})
            
        

### Turn network dictionary into Json file

In [94]:
# Serializing Json  
json_object = json.dumps(Network, indent = 4) 
with open("network/network_Amsterdam_scenario.json", "w") as outfile:
    outfile.write(json_object)

# Preparation for drawing links and nodes on ArcMap 

## Get coordinates of nodes in the network

In [96]:
# finction takes the filename of network Json file as input
# returns the dataframe contains all nodes' coordinates 
def nodesInJson(filename):
    # Opening JSON file
    f = open(filename)
    # returns JSON object as a dictionary
    network = json.load(f)
    # Iterating through the json
    # get nodes
    layers = network['LAYERS']
    nodes = layers[0]['NODES']  # a liste of dict
    ID = []
    X = []
    Y = []
    for dict in nodes:
        latlon = utm.to_latlon(dict['X'], dict['Y'], 31, 'U')
        ID.append(dict['ID'])
        X.append(latlon[0])
        Y.append(latlon[1])
    coord = list(zip(ID, X, Y))
    nodeDf = pd.DataFrame(coord, columns=['ID', 'X', 'Y'])
    return nodeDf 

In [95]:
nodeDf = nodesInJson('network/network_Amsterdam_scenario.json')
nodeDf

,ID,X,Y
0,m1000680335,52.646034,4.737600
1,m1000680344,52.646028,4.738807
2,m1000830542,52.362739,4.787499
3,m1003028149,52.363145,4.804220
4,m1003028151,52.362966,4.804315
...,...,...,...
17430,m997413137,52.351959,4.737111
17431,m997413138,52.352931,4.737559
17432,m997996379,52.357997,4.778998
17433,m997996404,52.357267,4.777355


### Get coordinates of links in the network 

In [97]:
def linksInJson(filename):
    # Opening JSON file
    f = open(filename)
    # returns JSON object as a dictionary
    network = json.load(f)
    # Iterating through the json
    # get links
    layers = network['LAYERS']
    links = layers[0]['LINKS'] 
    
    X_START = []
    Y_START = []
    X_END = []
    Y_END = []
    ID = []
    for dict in links:
        id = dict['ID']
        id_up = dict['UPSTREAM']
        id_down = dict['DOWNSTREAM']
        upDf = nodeDf.loc[(nodeDf.ID == id_up)]
        downDf = nodeDf.loc[(nodeDf.ID == id_down)]
        lat_s = upDf.iloc[0]['X']
        lon_s = upDf.iloc[0]['Y']
        lat_e = downDf.iloc[0]['X']
        lon_e = downDf.iloc[0]['Y']
        X_START.append(lat_s)
        Y_START.append(lon_s)
        X_END.append(lat_e)
        Y_END.append(lon_e)
        ID.append(id)
    coord = list(zip(ID, X_START, Y_START, X_END, Y_END))
    linkDf = pd.DataFrame(coord, columns=['ID', 'X_START', 'Y_START', 'X_END', 'Y_END'])
    return linkDf

In [98]:
linkDf = linkInJson('network/network_Amsterdam_scenario.json')
linkDf

,ID,X_START,Y_START,X_END,Y_END
0,m100106293_0,52.118519,5.034389,52.118749,5.034602
1,m100128682_0,52.356219,4.827185,52.356221,4.827275
2,m100128682_0_r,52.356221,4.827275,52.356219,4.827185
3,m100699928_0,52.118234,5.034047,52.118382,5.034090
4,m100699931_0,52.119122,5.034530,52.118653,5.034120
...,...,...,...,...,...
31127,m99485408_2,52.351341,4.856097,52.351375,4.856210
31128,m99485410_0,52.351004,4.851695,52.350951,4.852008
31129,m99485410_1,52.350951,4.852008,52.350931,4.852195
31130,m99485412_0,52.350992,4.852565,52.351032,4.852224


### Save dataframe as csv file

In [73]:
#linkDf.to_csv('linkDf.csv', sep = ";", index=False)
#nodeDf.to_csv('nodeDf.csv', sep = ";", index = False)